In [37]:
import numpy as np
import pandas as pd
from Utils import *
import re
from tqdm import tqdm
#crawler
import requests
from bs4 import BeautifulSoup

In [2]:
# 讀入官方的data
df = pd.read_csv('./data/tbrain_train_final_0610.csv')  

In [3]:
df[0:10]

,news_ID,hyperlink,content,name
0,1,https://news.cnyes.com/news/id/4352432,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...,[]
1,2,https://udn.com/news/story/120775/4112519,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ### 省略內文 ### 北...,[]
2,3,https://www.mirrormedia.mg/story/20191008soc011/,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文...,[]
3,4,https://www.chinatimes.com/realtimenews/201909...,58歲林姓男子昨凌晨與朋友聚餐結束後 ### 省略內文 ### 而地點就距離林家僅30公尺。...,[]
4,5,http://domestic.judicial.gov.tw/abbs/wkw/WHD9H...,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...,[]
5,6,http://www.coolloud.org.tw/node/93356,內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區 ### 省略內文 ### 市區拒...,[]
6,7,https://m.ctee.com.tw/livenews/gj/ctee/a076720...,華為創辦人任正非接受英國媒體訪問時表示 ### 省略內文 ### 重申中國和加拿大兩國之間的...,[]
7,8,https://www.mirrormedia.mg/story/20191209edi009/,2019.12.09 08:00「再教育營學員全已結業」新疆自治區主席：他們過上了幸福生活文...,[]
8,9,https://www.chinatimes.com/newspapers/20190827...,交通部觀光局26日公布7月國際旅客情況 ### 省略內文 ### 只能長期培養。,[]
9,10,https://news.cnyes.com/news/id/4365094,0理財基金基金汰弱留強該怎麼做？鉅亨買基金2019/08/06 21:30facebook ...,[]


In [4]:
# 取出網站種類
webKind = df["hyperlink"].str.split('/').str.get(2)
webKind = np.unique(webKind)
webKind

array(['ccc.technews.tw', 'domestic.judicial.gov.tw', 'ec.ltn.com.tw',
       'ent.ltn.com.tw', 'estate.ltn.com.tw', 'finance.technews.tw',
       'hk.on.cc', 'house.ettoday.net', 'm.ctee.com.tw', 'm.ltn.com.tw',
       'money.udn.com', 'mops.twse.com.tw', 'news.cnyes.com',
       'news.ebc.net.tw', 'news.ltn.com.tw', 'news.mingpao.com',
       'news.tvbs.com.tw', 'ol.mingpao.com', 'sina.com.hk', 'technews.tw',
       'tw.news.yahoo.com', 'udn.com', 'www.bnext.com.tw',
       'www.businesstoday.com.tw', 'www.chinatimes.com', 'www.cna.com.tw',
       'www.coolloud.org.tw', 'www.cw.com.tw', 'www.ettoday.net',
       'www.fsc.gov.tw', 'www.hbrtaiwan.com', 'www.hk01.com',
       'www.managertoday.com.tw', 'www.mirrormedia.mg',
       'www.nextmag.com.tw', 'www.nownews.com', 'www.setn.com',
       'www.storm.mg', 'www.wealth.com.tw'], dtype=object)

In [5]:
# 單獨測試
index = 2
url = df["hyperlink"][index]
category = getWebsiteCategory(url)
#context = getContext(category, url) 
context  = 0
print("url :", url, "\ncategory :", category, "\ncontext :", context)

url : https://www.mirrormedia.mg/story/20191008soc011/ 
category : www.mirrormedia.mg 
context : 0


In [46]:
def getContext(url):
    print("==========================")
    print("getContext url: ", url)
    soup, status_code = getSoup(url)
    contextStr = ""
    if status_code != 200:
        print("status_code: ", status_code, "url: ", url)
        contextStr = str(status_code)
    else:
        data = soup.find_all('p')
        for context in data:
            #print(context.text)
            contextStr += context.text
    return contextStr

In [47]:
context = getContext(url)

getContext url:  https://news.cnyes.com/news/id/4365094


In [57]:
df_addContext = df.copy()

In [55]:
# 原本想用lambda 不過這樣爬蟲會爬太快
#df_addContext['contextCrawler'] = df_addContext['hyperlink'].apply(lambda x: getContext(x))
contextList = []
hyperlinkList = df_addContext['hyperlink']
for index, url in enumerate(tqdm(hyperlinkList)):
    context = getContext(url)
    contextList.append(context)
df_addContext.insert(-1, "contextCrawler", contextList) 

0%|          | 0/10 [00:00<?, ?it/s]==========================
getContext url:  https://news.cnyes.com/news/id/4352432
 10%|█         | 1/10 [00:00<00:06,  1.50it/s]==========================
getContext url:  https://udn.com/news/story/120775/4112519



ValueError: cannot insert contextCrawler, already exists

In [56]:
df_addContext[:20]
dt.to_csv('./data/Result.csv')

,contextCrawler,news_ID,hyperlink,content,name
0,熱搜：熱門行情最近搜尋基金老司機近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易...,1,https://news.cnyes.com/news/id/4352432,0理財基金量化交易追求絕對報酬有效對抗牛熊市鉅亨網記者鄭心芸2019/07/05 22:35...,[]
1,熱搜：熱門行情最近搜尋基金老司機近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易...,2,https://udn.com/news/story/120775/4112519,10月13日晚間發生Uber Eats黃姓外送人員職災死亡案件 ### 省略內文 ### 北...,[]
2,熱搜：熱門行情最近搜尋基金老司機近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易...,3,https://www.mirrormedia.mg/story/20191008soc011/,2019.10.08 01:53【法拍有詭4】飯店遭管委會斷水斷電員工怒吼：生計何去何從？文...,[]
3,熱搜：熱門行情最近搜尋基金老司機近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易...,4,https://www.chinatimes.com/realtimenews/201909...,58歲林姓男子昨凌晨與朋友聚餐結束後 ### 省略內文 ### 而地點就距離林家僅30公尺。...,[]
4,熱搜：熱門行情最近搜尋基金老司機近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易...,5,http://domestic.judicial.gov.tw/abbs/wkw/WHD9H...,例稿名稱：臺灣屏東地方法院公示催告公告發文日期：中華民國108年9月20日發文字號：屏院進家...,[]
5,熱搜：熱門行情最近搜尋基金老司機近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易...,6,http://www.coolloud.org.tw/node/93356,內政部都市計畫委員會委員為審查大社工業區是否降為乙種工業區 ### 省略內文 ### 市區拒...,[]
6,熱搜：熱門行情最近搜尋基金老司機近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易...,7,https://m.ctee.com.tw/livenews/gj/ctee/a076720...,華為創辦人任正非接受英國媒體訪問時表示 ### 省略內文 ### 重申中國和加拿大兩國之間的...,[]
7,熱搜：熱門行情最近搜尋基金老司機近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易...,8,https://www.mirrormedia.mg/story/20191209edi009/,2019.12.09 08:00「再教育營學員全已結業」新疆自治區主席：他們過上了幸福生活文...,[]
8,熱搜：熱門行情最近搜尋基金老司機近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易...,9,https://www.chinatimes.com/newspapers/20190827...,交通部觀光局26日公布7月國際旅客情況 ### 省略內文 ### 只能長期培養。,[]
9,熱搜：熱門行情最近搜尋基金老司機近年來投資市場波動越來越明顯，追求低波動、絕對報酬的量化交易...,10,https://news.cnyes.com/news/id/4365094,0理財基金基金汰弱留強該怎麼做？鉅亨買基金2019/08/06 21:30facebook ...,[]
